In [2]:
from pathlib import Path
from tqdm import tqdm
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import math
import numpy as np
import pandas as pd
import os
import json
import pickle
import glob
import torch

opj = lambda x, y: os.path.join(x, y)

In [24]:
data_path = Path('/common/users/dm1487/milnce_features')
ann_path = Path('/common/users/dm1487/YouCookII')
videos_path = data_path/'raw_frames'
steps_path = data_path/'raw_text'
videos_path

PosixPath('/common/users/dm1487/milnce_features/raw_frames')

In [27]:
image_features = glob.glob(str(videos_path/'training/*/*.pth'))
text_features = glob.glob(str(steps_path/'training/*/*.pth'))
image_keys, text_keys = [i.split('/')[-1] for i in image_features], [i.split('/')[-1] for i in text_features]
image_df = pd.DataFrame({'image_features': image_features, 'image_keys': image_keys}).set_index('image_keys')
text_df = pd.DataFrame({'text_features': text_features, 'text_keys': text_keys}).set_index('text_keys')
all_data = image_df.join(text_df)
dataset = [(img, txt) for img, txt in zip(all_data['image_features'].tolist(), all_data['text_features'].tolist())]

In [29]:
all_data

,image_features,text_features
image_keys,,
uek-TzOhu9I.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
jGnz9f_2A2Y.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
sQl3bQ1rQUM.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
H4-p87JS-yA.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
0q38LzW-axY.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
...,...,...
jpQBWsR3HHs.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
uI-vgcqOioU.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...
tasuUgO6m3c.pth,/common/users/dm1487/milnce_features/raw_frame...,/common/users/dm1487/milnce_features/raw_text/...


In [30]:
train_ds = []
valid_ds = []
for img, txt in dataset:
    if 'training' in img:
        train_ds.append((img, txt))
    if 'validation' in img:
        valid_ds.append((img, txt))

with open('dataset/training_data.pkl', 'wb') as f:
    pickle.dump(train_ds, f)

with open('dataset/validation_data.pkl', 'wb') as f:
    pickle.dump(valid_ds, f)

In [31]:
PATH = Path('data')
data = None
with open(PATH/'youcookii_annotations_trainval.json', 'rb') as f:
    data = json.load(f)
data['database']

{'GLd3aX16zBg': {'duration': 241.62,
  'subset': 'training',
  'recipe_type': '113',
  'annotations': [{'segment': [90, 102],
    'id': 0,
    'sentence': 'spread margarine on two slices of white bread'},
   {'segment': [114, 127],
    'id': 1,
    'sentence': 'place a slice of cheese on the bread'},
   {'segment': [132, 138],
    'id': 2,
    'sentence': 'place the bread slices on top of each other and place in a hot pan'},
   {'segment': [139, 145],
    'id': 3,
    'sentence': 'flip the sandwich over and press down'},
   {'segment': [173, 174],
    'id': 4,
    'sentence': 'cut the sandwich in half diagonally'}],
  'video_url': 'https://www.youtube.com/watch?v=GLd3aX16zBg'},
 'xHr8X2Wpmno': {'duration': 206.86,
  'subset': 'validation',
  'recipe_type': '226',
  'annotations': [{'segment': [47, 60],
    'id': 0,
    'sentence': 'pick the ends off the verdalago'},
   {'segment': [67, 89],
    'id': 1,
    'sentence': 'combine lemon juice sumac garlic salt and oil in a bowl'},
   {'se

In [32]:
step_text = []
for k, v in data['database'].items():
    steps = []
    for ann in v['annotations']:
        steps.append(ann['sentence'])
    step_text.append({
        'name': k, 
        'subset': v['subset'],
        'recipe_type': v['recipe_type'],
        'sentences': steps
    })
with open(PATH/'step_texts.pkl', 'wb')  as f:
    pickle.dump(step_text, f)

In [33]:
DATA_PATH = Path('/common/users/dm1487/YouCookII')
os.listdir(DATA_PATH/'features/feat_csv/train_frame_feat_csv/101/ucky1nWb7LY/0001')
DATA_DIR = DATA_PATH/'features/feat_csv/train_frame_feat_csv/101/ucky1nWb7LY/0001'
df = pd.read_csv(DATA_DIR/'resnet_34_feat_mscoco.csv', header=None)

image_features = np.array([df.iloc[i].tolist() for i in range(df.shape[0])])
image_features.shape

(500, 512)

In [34]:
text_data_file = PATH/'youcook_text.csv'
df_text = pd.read_csv(text_data_file)
df_text[df_text['key'] == 'ucky1nWb7LY'].index 

Int64Index([808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818], dtype='int64')

In [36]:
with open(PATH/'features_text_instructions/all_text_features.pkl', 'rb') as f:
    text_features = pickle.load(f)
# text_features = torch.load(PATH/'features_text_instructions/all_text_features.pkl')
# text_features[df_text[df_text['key'] == 'ucky1nWb7LY'].index].shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/features_text_instructions/all_text_features.pkl'

In [ ]:
pca = PCA(512)
# print(text_features.cpu().numpy())
text_features_dr = torch.tensor(pca.fit_transform(text_features.cpu().numpy()))

In [ ]:
text_features_dr.size()

In [ ]:
mat = text_features_dr[df_text[df_text['key'] == 'ucky1nWb7LY'].index] @ image_features.T

In [ ]:
# torch.softmax(mat, dim = 0)
val, ind = torch.topk(mat, dim=1, k=10)
torch.sort(ind,dim=1)

In [ ]:
sorted(os.listdir(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001'))

np.loadtxt(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001/resnet_34_feat_mscoco.dat')

In [ ]:
sorted(os.listdir(PATH/'features'/'feat_csv/train_frame_feat_csv/101'))

In [ ]:
df = pd.read_csv(PATH/'features'/'feat_csv/train_frame_feat_csv/113/2vNPfc8LaTc/0010/resnet_34_feat_mscoco.csv', header=None)
df.shape

In [37]:
# train_split = None
# with open(PATH/'splits/train_list.txt', 'r') as f:
#     train_split = f.readlines()
# train_split
os.listdir(PATH/'splits')

['val_list.txt',
 'test_list.txt',
 'train_duration_totalframe.csv',
 'val_duration_totalframe.csv',
 'train_list.txt',
 'test_duration_totalframe.csv']

In [42]:
df_linker = pd.read_csv(PATH/'splits'/'train_duration_totalframe.csv')
# df_linker[df_linker['vid_id'] == '2vNPfc8LaTc']
df_linker['I'] = (df_linker['total_frame']/500).apply(math.ceil)
df_linker['fps'] = (df_linker['total_frame']/df_linker['duration']).apply(math.ceil)
df_linker['spf'] = (1/df_linker['fps'])
df_linker.head()

,vid_id,duration,total_frame,I,fps,spf
0,-xbTvALWCIg,455.02,10908,22,24,0.041667
1,8X2wDyjl6T8,46.00,1377,3,30,0.033333
2,bczCfVC6HYU,201.42,5035,11,25,0.040000
3,aMtZQgH_MRc,163.02,4889,10,30,0.033333
4,RV0_DWkbjx8,135.88,4072,9,30,0.033333


In [43]:
# df.set_index('key').join(df_linker.set_index('vid_id')).head(20)

In [44]:
# df_linker['frames_needed'] = [None] * df_linker.shape[0]
frames_disected = []
for idx, row in df_linker.iterrows():
    frames_needed = []
    frames_repre = []
    m = 0
    while m < row['total_frame']:
        frames_needed.append(m)
        frames_repre.append(m * row['spf'])
        m += row['I']
#     print(frames_needed)
    frames_disected.append([row['vid_id'], frames_needed.copy(), frames_repre.copy()])
#     break
# df_linker.head()
# df_linker['frames_needed'] = df_linker['fps']/df_linker['I']

In [40]:
df_segments = pd.DataFrame(frames_disected, columns=['vid_id', 'frames_recorded', 'segments']).set_index('vid_id').join(df.set_index('key'))
df_segments['segment'] = df_segments['segment'].apply(eval)
df_segments.head()

KeyError: "None of ['key'] are in the columns"

In [ ]:
frames_needed_for_extract = []
for idx, row in df_segments.iterrows():
    start, end = tuple(row['segment'])
    arr = np.array(row['segments'])
    arg_index = np.argwhere((arr >= start) & (arr < end))
    frames_needed_for_extract.append([idx, arg_index.flatten().tolist()])

df_extract = pd.DataFrame(frames_needed_for_extract, columns=['vid_id', 'extract'])

In [ ]:
df.set_index('key').join(df_extract.set_index('vid_id'))

In [ ]:
all_df = df.set_index('key').join(df_linker.set_index('vid_id'))

In [ ]:
all_df = all_df[all_df['subset'] == 'training']


In [ ]:
all_df['segment']